# Plot Loop-Closure-Detection

Plots statistics on loop closure detection as well as optimized trajectory RPE, APE and trajectory against ground truth.

In [1]:
import yaml
import os
import copy
import pandas as pd
import numpy as np

import logging
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
if not log.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(logging.Formatter('%(levelname)s - %(message)s'))
    log.addHandler(ch)
    
from evo.tools import file_interface
from evo.tools import plot
from evo.tools import pandas_bridge

from evo.core import sync
from evo.core import trajectory
from evo.core import metrics
from evo.core import transformations
from evo.core import lie_algebra as lie

%matplotlib inline
# %matplotlib notebook
import matplotlib.pyplot as plt

## Data Locations

Make sure to set the following paths.

`vio_output_dir` is the path to the directory containing `output_*.csv` files obtained from logging a run of SparkVio.

`gt_data_file` is the absolute path to the `csv` file containing ground truth data for the absolute pose at each timestamp of the dataset.

In [2]:
# Define directory to VIO output csv files as well as ground truth absolute poses.
# vio_output_dir = "/home/marcus/code/VIO/output_logs/test_3/"
vio_output_dir = "/home/marcus/catkin_ws/src/spark_vio_ros/output_logs/test_4/"
# gt_data_file = "/home/marcus/datasets/V1_01_easy/mav0/state_groundtruth_estimate0/data.csv"
gt_data_file = "/home/marcus/catkin_ws/src/spark_vio_ros/output_logs/test_4/output_gt_poses.csv"

In [3]:
def get_ape(data, metric):
    """ Gets APE and APE statistics for two trajectories and a given pose_relation.
        
        Args:
            data:   tuple of trajectories, the first being the reference trajectory
                    and the second being the estimated trajectory.
            metric: a metrics.PoseRelation instance representing the pose relation
                    to use when computing APE.
        
        Returns:
            A metrics.APE instance containing the APE for both trajectories according
            to the given metric.
    """
    ape = metrics.APE(metric)
    ape.process_data(data)
    
    return ape


def plot_ape(x_axis, ape, size=(18,10)):
    """ Plots APE error against time for a given metrics.APE instance.
    
        Args:
            x_axis: An array-type of values for all the x-axis values (time).
            rpe:    A metrics.APE instance with pre-processed data.
            size:   A tuple optionally containing the size of the figure to be plotted.
    """
    fig = plt.figure(figsize=size)
    plot.error_array(fig, ape.error, x_array=x_axis, statistics=ape.get_all_statistics(), 
                 name="APE", title="APE w.r.t. " + ape.pose_relation.value, xlabel="$t$ (s)")
    plt.show()


def get_rpe(data, metric):
    """ Gets RPE and RPE statistics for two trajectories and a given pose_relation.
        
        Args:
            data:   tuple of trajectories, the first being the reference trajectory
                    and the second being the estimated trajectory.
            metric: a metrics.PoseRelation instance representing the pose relation
                    to use when computing RPE.
        
        Returns:
            A metrics.RPE instance containing the RPE for both trajectories according
            to the given metric.
    """
    # normal mode
    delta = 1
    delta_unit = metrics.Unit.frames
    all_pairs = False

    rpe = metrics.RPE(metric, delta, delta_unit, all_pairs)
    rpe.process_data(data)
    return rpe


def plot_rpe(x_axis, rpe, size=(18,10)):
    """ Plots RPE error against time for a given metrics.RPE instance.
    
        Args:
            x_axis: An array-type of values for all the x-axis values (time).
            rpe:    A metrics.RPE instance with pre-processed data.
            size:   A tuple optionally containing the size of the figure to be plotted.
    """
    fig = plt.figure(figsize=size)
    plot.error_array(fig, rpe.error, x_array=x_axis, statistics=rpe.get_all_statistics(), 
                     name="RPE", title="RPE w.r.t. " + rpe.pose_relation.value, xlabel="$t$ (s)")
    plt.show()


def rename_euroc_gt_df(df):
    """ Renames a DataFrame built from a EuRoC ground-truth data csv file to be easier to read.
        
        Column labels are changed to be more readable and to be identical to the generic pose 
        trajectory format used with other csv files. Note that '#timestamp' will not actually 
        be renamed if it is the index of the DataFrame (which it should be). It will be 
        appropriately renamed if it is the index name.
        This operation is 'inplace': It does not return a new DataFrame but simply changes
        the existing one.
        
        Args:
            df: A pandas.DataFrame object.
    """
    df.index.names = ["timestamp"]
    df.rename(columns={" p_RS_R_x [m]": "x",
                       " p_RS_R_y [m]": "y",
                       " p_RS_R_z [m]": "z",
                       " q_RS_w []": "qw",
                       " q_RS_x []": "qx",
                       " q_RS_y []": "qy",
                       " q_RS_z []": "qz",
                       " v_RS_R_x [m s^-1]": "vx",
                       " v_RS_R_y [m s^-1]": "vy",
                       " v_RS_R_z [m s^-1]": "vz",
                       " b_w_RS_S_x [rad s^-1]": "bgx",
                       " b_w_RS_S_y [rad s^-1]": "bgy",
                       " b_w_RS_S_z [rad s^-1]": "bgz",
                       " b_a_RS_S_x [m s^-2]": "bax",
                       " b_a_RS_S_y [m s^-2]": "bay",
                       " b_a_RS_S_z [m s^-2]": "baz"}, inplace=True)

## LoopClosureDetector Statistics Plotting

Gather and plot various statistics on LCD module performance, including RANSAC information, keyframe status (w.r.t. loop closure detection), and loop closure events and the quality of their relative poses.